task name: Implement WithOnFitDelegate
task description: Implement the provided C# code using dotnet interactive notebook

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Collections.Generic;
using System.Collections.Immutable;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;
using static Microsoft.ML.Transforms.NormalizingTransformer;


In [ ]:
public class DataPoint
{
    [VectorType(4)]
    public float[] Features { get; set; }
    public bool Label { get; set; }
}

var mlContext = new MLContext();
var samples = new List<DataPoint>()
{
    new DataPoint(){ Features = new float[4] { 8, 1, 3, 0}, Label = true },
    new DataPoint(){ Features = new float[4] { 6, 2, 2, 0}, Label = true },
    new DataPoint(){ Features = new float[4] { 4, 0, 1, 0}, Label = false },
    new DataPoint(){ Features = new float[4] { 2,-1,-1, 1}, Label = false }
};

In [ ]:
var data = mlContext.Data.LoadFromEnumerable(samples);

In [ ]:
NormalizingTransformer binningTransformer = null;
var pipeline =
    mlContext.Transforms
    .NormalizeBinning("Features", maximumBinCount: 3)
    .WithOnFitDelegate(
    fittedTransformer => binningTransformer = fittedTransformer)
    .Append(mlContext.BinaryClassification.Trainers
    .LbfgsLogisticRegression());

In [ ]:
var model = pipeline.Fit(data);
var binningParam = binningTransformer.GetNormalizerModelParameters(0) as
    BinNormalizerModelParameters<ImmutableArray<float>>;
for (int i = 0; i < binningParam.UpperBounds.Length; i++)
{
    var upperBounds = string.Join(", ", binningParam.UpperBounds[i]);
    Console.WriteLine(
        $"Bin {i}: Density = {binningParam.Density[i]}, " +
        $"Upper-bounds = {upperBounds}");
}